<a href="https://colab.research.google.com/github/eliottpark/BIOE145/blob/main/MultipleEM_Motifs_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple EM for Motif Elicitation

In [ ]:
%pip install biopython

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize
from Bio import SeqIO

In [ ]:
# Constant used in this exercise
# Fill in all of the ...s/TODOs
width = 8

# Helper dict for indexing
let_dict = {"A":0, "C":1, "G":2, "T":3}

## 1. Read in Fasta Sequences

In [ ]:
# Read in the files using BioPython
# TODO:
records = [] # as Seq record objects
for record in SeqIO.parse("motif-regions.fa", "fasta"):
  records.append(record)

In [ ]:
# Extract strings of sequences from the above files
# TODO:

sequences = []
for record in records:
  sequences.append(str(record.seq))

## 2. Create p_0

In [ ]:
# Initialize p with a uniform background

def init_p(l, w, seqs, let):
    """
    Initialize standing probabilites.
    l: sequence length
    w: motif length
    let: letter dictionary
    """
    p = np.zeros((4, w+1))

    # set a uniform background for each A C G T
    # The 0th index represents background probabilites
    # TODO:
    for i in range(len(let)):
        # TODO:
        p[i][0] = 0.25

    # set motif positions
    for i in range(l-w+1):
        for sequence in seqs:
            for j in range(w):
                # Fill in p_0
                # TODO:
                p[let[sequence[i+j]]][j+1] += 1
                
    # normalize columns to sum to 1
    p = normalize(p, axis = 0, norm = 'l1')
    
    return p

## 3. Fill in EM iteration

In [ ]:
# Define a general function to run EM

def run_EM(w, seqs, let, init_p, up_prob, up_motif, epsilon = 0.0001):
    
    l = len(seqs[0])
    
    no_change = False
    
    # set an initial p_t_1
    # TODO:
    p_t_1 = init_p(l, width, seqs, let)
        
    while not no_change:
        
        # Label the following steps as E step or M step in the comment preceding
        
        # TODO:
        # E Step
                
        z_t = up_motif(l, w, p_t_1, seqs, let)
        
        # TODO:
        # Maximization
                
        p_t = up_prob(l, w, z_t, seqs, let)
        
        diff = np.subtract(p_t, p_t_1)
        
        # Write a condition to stop the EM iterations (use epsilon and diff)
        # TODO:
        if np.all(diff < epsilon):        
            no_change = True
        else:
            # Update p_t_1
            # TODO:
            p_t_1 = p_t
                
    return p_t, z_t

## 4. Fill in function to update z_t

In [ ]:
# Define a function to update z

def up_motif(l, w, p_t_1, seqs, let):
    z_t = np.zeros((len(seqs), l-w+1))

    for i, sequence in enumerate(seqs):
        for j in range(l-w+1):
            
            # Fill in z_t using p_t_1
            # Ignore background as we're assuming 0.25 for all 4
            # TODO:
            z_t[i][j] = 1/(l-w+1)
            for k in range(w):
              z_t[i][j] *= p_t_1[let[sequence[j+k]]][k+1]
    
    # Normalize z_t
    # TODO:
    z_t = normalize(z_t, axis = 1, norm = 'l1')
        
    return z_t
    

## 5. Fill in function to update p_t

In [ ]:
# Define a function to update p

def up_prob(l, w, z_t, seqs, let):
    p_t = np.zeros((4, w+1))
    
    n = np.zeros((4, w+1))
    
    # Fill in n for k > 0
    for k in range(1, w+1):
        for letter in let.keys():
            sum_z = 0
            for i, sequence in enumerate(seqs):
                
                # Write j_vals according to the condition seen in lecture
                # TODO:
                for j in range(l-w+1):
                  if sequence[j+k-1] == letter:
                    sum_z += z_t[i][j]
            
            # Fill in the correct indices
            # TODO:
            n[let[letter]][k] = sum_z
                 
    # Fill in n for k == 0
    
    # May help to make the next step easier
    joined_seq = "".join(seqs)
    
    # Create a dict with total counts of A,C,G,T
    # TODO:
    counts = {c:joined_seq.count(c) for (c,v) in let_dict.items()}
        
    # Sum across the rows of n
    # TODO:
    sum_n_j = np.sum(n, axis=1)
        
    for i, letter in enumerate(let.keys()):
        
        # Fill in the correct indices and its value
        # TODO:
        n[let[letter]][0] = counts[letter] - sum_n_j[i]

    
    # Use n to fill in p_t
    # Pseudo-count = 1
    # TODO:
    denom = np.sum(n, axis=0)
    for k in range(w+1):
      for letter in let.keys():
        p_t[let[letter]][k] = (n[let[letter]][k] + 1)/ (denom[k] + 4)
        
    return p_t
    

## 6. Run the EM to find the final p and z

In [ ]:
# Use the variables set at the start and 
# TODO:
p_end, z_end = run_EM(width, sequences, let_dict, init_p, up_prob, up_motif)

## 7. Determine Motifs

In [ ]:
# Find the indices of the max element for each row in z_end
# TODO:
motif_indices = np.argmax(z_end, axis=1)

# Get the 'width' characters long motifs using seqs
# TODO:
motifs = [sequences[i][k:k+width] for (i,k) in enumerate(motif_indices)]
print(motifs)

['AGAAAAAT', 'TGTAGATT', 'AATACAAT', 'GGAAAATT', 'AGAAGATT', 'AGAAAAAA', 'AGAACAAT', 'AGAAAAAA', 'AGAATATT', 'AAAAAATT', 'AGAAAAAA', 'AGAAAAAA', 'AAAAAAAT', 'AGAATATT', 'GGAAGGTT', 'AGAACATT', 'AGAAAATG', 'AGAAAAAA', 'GCTAAATT', 'AAAAAAAT', 'GGAAAAAG', 'AGAACCAT', 'GGAAATTT', 'ACAACAAG', 'AAAAAAAT', 'ACAAAATT', 'AGAAAAAT', 'AGAAAAAT', 'AAAAGAAT', 'AGAAATTT', 'GGAAAAAT', 'CGAAAATT', 'GGAAAATT', 'AAAAGATT', 'AGAAAAAA', 'GGAAGAAA', 'AGAAGAAA', 'AAAAGATT', 'AATAGAAT', 'GGAAAAAT', 'AAAAAAAT', 'AGAATAAT', 'AGAACAAT', 'AAAAGCTG', 'AGAAAAAT', 'AGAAAAAT', 'GGAAAATC', 'GGAAAATT', 'AAAAAAAT', 'GCTAGATT']
